In [ ]:
import re
from lxml import etree
from glob import glob
from os import path
from pprint import pprint

from tf.fabric import Timestamp

tm = Timestamp()

REPO = '~/github/pthu/patristics'
SRC_DIR = SRC_DIR = path.expanduser(f'{REPO}/sources') #/sourcetexts')

authorRE = re.compile(r'<author>|<author .+?>') #[^>]*?(.+)</author>')
editorRE = re.compile(r'<editor>|<editor .+?>') #[^>]*?(.+)</editor>')
titleRE = re.compile(r'<title>|<title .+?>')    #[^>]*?(.+)</title>')
bodyRE = re.compile(r'</*body>|</*body .+?>') 

# RE PATTERNS BODY
openTagRE = re.compile(r'<[^/= ]+?>')
closeTagRE = re.compile(r'</.+?>')
opencloseTagRE = re.compile(r'<[^=/]+?/>')
openAttrTagRE = re.compile(r'<.+?=.*?[^/]>')
closedAttrTagRE = re.compile(r'<.+?=.*?/>')
commentFullRE = re.compile(r'^<!--.*?-->$')
commentStartRE = re.compile(r'^<!--.*')
commentStopRE = re.compile(r'.*-->$')

result_set = set()
COUNTER = 0
COUNTER2 = 0

def authorWork(path):
    author = None
    editor = None
    book = None
    afound = False
    efound = False
    bfound = False
    with open(path, encoding='utf-8') as xml:
        for x in xml:
            line = x.strip()
            if authorRE.search(line) and afound == False:
                author = ''.join(re.split(r"<[^>]+?>", line)).strip('{ ,.}')
                afound = True
            if editorRE.search(line) and efound == False:
                editor = ''.join(re.split(r"<[^>]+?>", line)).strip('{ ,.}')
                efound = True
            if titleRE.search(line) and bfound == False:
                book = ''.join(re.split(r"<[^>]+?>", line)).strip('{ ,.}')
                bfound = True
            if bodyRE.search(line):
                if author == None:
                    author = editor
                break
    result_set.add((author, book))
#     return (author work)

def refsDecl(path):
    global COUNTER2
    with open(path, encoding='utf-8') as xml:
        found = False
        
        for x in xml:
            line = x.strip()
            if found == True:
                print(line)
            if re.search(r'<refsDecl', line):
                found = True
                print(line)
            if re.search(r'</refsDecl', line):
                print(line)
                break
        if found == False:
            result_set.add(path)
            COUNTER2 +=1

def cRefPattern(path):
    global COUNTER2
    with open(path, encoding='utf-8') as xml:
        found = False
        
        for x in xml:
            line = x.strip()
            if found == True:
                print(line)
            if re.search(r'<cRefPattern', line):
                found = True
                print(line)
            if re.search(r'</cRefPattern', line):
                print(line)
                break
            if re.search(r'</refsDecl', line):
                break
        if found == False:
            result_set.add(path)
            COUNTER2 +=1
            
def state(path):
    global COUNTER2
    with open(path, encoding='utf-8') as xml:
        found = False
        
        for x in xml:
            line = x.strip()
            if found == True:
                print(line)
            if re.search(r'<state', line):
                found = True
                print(line)
            if re.search(r'</state', line):
                print(line)
                break
            if re.search(r'</refsDecl', line):
                break
        if found == False:
            result_set.add(path)
            COUNTER2 +=1
            
def step(path):
    global COUNTER2
    with open(path, encoding='utf-8') as xml:
        found = False
        
        for x in xml:
            line = x.strip()
            if found == True:
#                 tag_split = line.find(" ")
                result_set.add(line)
                result_set.add(path)
            if re.search(r'<step', line):
                found = True
                result_set.add(line)
            if re.search(r'</refsDecl', line):
                break
        if found == False:
#             result_set.add(path)
            COUNTER2 +=1
            
def findRefDecl(path):
    global COUNTER2
    with open(path, encoding='utf-8') as xml:
        found = False
        
        for x in xml:
            line = x.strip()
            if found == True:
#                 tag_split = line.find(" ")
                result_set.add(line)
            if re.search(r'<refsDecl', line):
                found = True
#                 print(line)
            if re.search(r'</refsDecl', line):
                break
        if found == False:
            result_set.add(path)
            COUNTER2 +=1

def readfile(path):
    with open(path) as xml:
    # 16 sec
#         data = [line.strip().split() for line in xml.readlines()]
    # 2,89 sec
#         data = xml.readlines()
    # 3,34 sec
#         data = xml.read()
    # 5,99 sec
#         data = xml.readlines().replace('\n', ' ').replace("<", "#!#<").replace(">", ">#!#").split("#!#")
    # 7,25 sec
        data = ' '.join([line.strip() for line in xml.readlines()])\
                        .replace("<", "#!#<")\
                        .replace(">", ">#!#")\
                        .split("#!#")
        
    # 17 sec
#         data = re.sub(r'\s+\s+', ' ', xml.read().replace('\n', ' ').replace("<", "#!#<").replace(">", ">#!#")).split("#!#")

def findDivs(path):
    global COUNTER2
    global result_set
    with open(path) as xml:
        data = ' '.join([line.strip() for line in xml.readlines()])\
                        .replace("<", "#!#<")\
                        .replace(">", ">#!#")\
                        .split("#!#")
        for elem in data:
            if elem.startswith('<div'):
                result_set.add(elem)
                COUNTER2 +=1
                
def findOpenTags(path):
    TagRE = re.compile(r'<[^/]+?>')
    global COUNTER2
    global result_set
    with open(path) as xml:
        data = ' '.join([line.strip() for line in xml.readlines()])\
                        .replace("<", "#!#<")\
                        .replace(">", ">#!#")\
                        .split("#!#")
        for elem in data:
            if TagRE.match(elem):
                name = elem.split(' ')[0]
                result_set.add(name)
                COUNTER2 +=1

for xmlfile in glob(SRC_DIR+'/**/*grc*.xml', recursive=True):
    COUNTER +=1
#     tm.info(f'parsing {xmlfile}')
#     authorWork(xmlfile)
#     refsDecl(xmlfile)
#     cRefPattern(xmlfile)
#     state(xmlfile)
#     findRefDecl(xmlfile)
#     step(xmlfile)
#     readfile(xmlfile)
#     findDivs(xmlfile)
    findOpenTags(xmlfile)
pprint(result_set)
tm.info('--> Total Time')
print(f'{COUNTER} works have been parsed...')
print(f'{COUNTER2} works show a problem...')

In [ ]:
from io import StringIO

string = StringIO('''                    <l n="1">
                  <said who="#Ἀμφιτρύων">Τίς τὸν Διὸς σύλλεκτρον οὐκ οἶδεν βροτῶν,</said>
               </l>
               <l n="2">
                  <said who="#Ἀμφιτρύων">Ἀργεῖον Ἀμφιτρύωνʼ, ὃν Ἀλκαῖός ποτε</said>
               </l>
               <l n="3">
                  <said who="#Ἀμφιτρύων">ἔτιχθʼ ὁ Περσέως, πατέρα τόνδʼ Ἡρακλέους;   </said>
               </l>
               <l n="4">
                  <said who="#Ἀμφιτρύων">ὃς τάσδε Θήβας ἔσχον, ἔνθʼ ὁ γηγενὴς</said>
               </l>
''')
data = ' '.join([line.strip() for line in string.readlines()])\
                        .replace("<", "#!#<")\
                        .replace(">", ">#!#")\
                        .split("#!#")
print(data)
lis = []
for elem in data:
    elem = elem.strip()
    if elem == '':
#     if bool(elem) == False:
        continue
    else: 
        lis.append(elem)
lis

In [ ]:
dictio = {'biblStruct': {'author': 'Galen', 
                         'title': 'On the Natural Faculties', 
                         'editor': 'J.K. Elliot', 
                         'publisher': 'Oxford University Press', 
                         'date': '2013'}
         }
description = ', '.join([v for k, v in dictio['biblStruct'].items()])
print(description)

for k, v in dictio['biblStruct'].items():
    if k.startswith('editor'):
        print(v)

In [ ]:
string = ['SourceDesc', 'Biblstmt']
for i in string:
    if 'ource' in i:
        print(i)

In [ ]:
import re

book = 'Description of Greece (Greek). Machine readable text (Greek)'
# book = 'Annotations. (Greek)'
print(book)
book = book.replace('(Greek)', '').replace('.', '').replace('Machine readable text', '')
# book = book.strip("('(Greek)', '.', ',', 'Machine Readible Text')")
# replace_all(book, {'Greek': ''})

print(book)

In [ ]:
elem = '<author ref="http://data.perseus.org/catalog/urn:cts:greekLit:tlg2200">Libanius</author>'
tag_split = elem.find(' ') if not elem.find(' ') == -1 else elem.find('>')
print(elem[1:tag_split])

In [ ]:
counter = dict(
    sentence=0,
    word=0,
    letter=15)

tag_name = 'letter'

print(counter)
counter[tag_name] = 1 if tag_name not in counter else counter[tag_name] + 1

print(counter)

In [ ]:
lis = ['2', '3', ' 56', '54', '45', '67', '3']
all(y.strip().isdigit() for y in lis)

for x.startswith('5') in lis:
    print(x)

In [ ]:
a = set()
lis = [1, 2, 3, 4]
a.update(lis)
print(a)

intFeatures = {'section', 'number'}

x = 'section'
y = 'chapter'

result = f'book, {"number" if x in intFeatures else "label"}'
print(result)


In [ ]:
from os import path

REPO = '~/github/pthu/patristics'
SRC_DIR = SRC_DIR = path.expanduser(f'{REPO}/sources/pt/tlg2021/tlg001/tlg2021.tlg001.opp-grc1.xml') 

class Data:
    def __init__(self, path):
        self.path = path
        self.data = self.dataFinder(self.path)
        
    def dataFinder(self, path):
        with open(path) as xml:
            data = ' '.join([line.strip() for line in xml.readlines()])\
                      .replace('<', '#!#<')\
                      .replace('>', '>#!#')\
                      .split('#!#')

        return data

x = Data(SRC_DIR)
print(x.data)

In [ ]:
dictio = {1: {1:1}, 2: {2:2}}
print(dictio)
dictio1 = {k: v for k,  in dictio.items() for key: len(value) in v.items()}
print(dictio1)

In [ ]:
import re
elem = '<gap reason  =  "                                 omitted"/>'
print(elem)
elem = re.sub(r'\s*=\s*"\s*', '="', elem)
print(elem)

In [ ]:
string = 'abcdefg'
if string.startswith(('b', 'a')):
    print("yes")

In [ ]:
from cltk.stem.lemma import LemmaReplacer
from cltk.lemmatize.greek.backoff import BackoffGreekLemmatizer

# lemmatizer = BackoffGreekLemmatizer()

sentence = 'τὰ γὰρ πρὸ αὐτῶν καὶ τὰ ἔτι παλαίτερα σαφῶς μὲν εὑρεῖν διὰ χρόνου πλῆθος ἀδύνατα ἦν'

lemmatizer = LemmaReplacer('greek')
lemmatizer.lemmatize(sentence, return_string=False)

In [ ]:
import betacode.conv

string = '''*polla/ me ta\ parakalou=nta/ e)sti cumbouleu=sai
u(mi=n, w)= pai=des, a(\ be/ltista ei)=nai kri/nw,
kai\ a(\ cunoi/sein u(mi=n e(lome/nois pepi/steuka. to/
te ga\r h(liki/as ou(/tws e)/xein kai\ to\ dia\ pollw=n
h)/dh gegumna/sqai pragma/twn kai\ mh\n kai\ to\
th=s pa/nta paideuou/shs e)p) a)/mfw metabolh=s
i(kanw=s metasxei=n, e)/mpeiro/n me ei)=nai tw=n a)nqrwpi/nwn
pepoi/hken, w(/ste toi=s a)/rti kaqistame/nois
to\n bi/on e)/xein w(/sper o(dw=n th\n a)sfalesta/thn
u(podeiknu/nai.'''

print(betacode.conv.beta_to_uni(string, strict=True))

string1 = 'τὰ γὰρ πρὸ αὐτῶν καὶ τὰ ἔτι παλαίτερα σαφῶς μὲν εὑρεῖν διὰ χρόνου πλῆθος ἀδύνατα ἦν'

string1.encode('ascii')

In [ ]:
string = 'Origin of Species. (Greek) Machine Readible Text'
string = string.strip("'(Greek)', '.', ',', 'Machine Readible Text'}")
print(string)

In [ ]:
st = 'pseudo-justinus martyr'
st.title()

In [ ]:
from unicodedata import normalize

ELISION = {'ἀλλ᾽': 'ἀλλὰ', 'ἀλλʼ': 'ἀλλὰ', 'ἐφ᾽': 'ἐπὶ', 'διʼ': 'διά', 'τ᾽': 'τέ', 'δ᾽': 'δέ', 'γ᾽': 'γέ', 'ὅτ᾽': 'ὅτε', 'ἐπ᾽': 'ἐπὶ', 'ἀνθ᾽': 'ἀντὶ', 'ὑπ᾽': 'ὑπο', 'ἔσθ᾽': 'ἐστὶν', 'οὐδ᾽': 'οὐδέ', 'ἀφ᾽': 'ἀπό', 'καθ᾽': 'κατά', 'μεθ᾽': 'μετά',}

ELISION1 = {normalize('NFC', k): normalize('NFC', v) for k, v in ELISION.items()}

print(ELISION1)

In [ ]:
st = ('milestone', ('unit', 'n', 'rend'))
if st[0].startswith(('div', 'milestone')):
    print("yes")
if all(i in st[1] for i in ('unit', 'n')):
    print("yes again!")

In [ ]:
lis = ['chapter']
lis1 = ('section', 'subsection')
lis.extend(lis1)
print(lis)
print(lis[1:])
lis.index('chapter')
print(lis[0:])

In [ ]:
lsi = []
if lsi:
    print("yes")

In [7]:
tup = (1, 3, 5)
print(".".join((str(i) for i in tup[:-1])))

1.3
